## For Colab Only

In [ ]:
%pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-8lp0455p
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-8lp0455p
  Resolved https://github.com/facebookresearch/detectron2.git to commit b7c7f4ba82192ff06f2bbb162b9f67b00ea55867
  Preparing metadata (setup.py) ... done


In [ ]:
!pip show detectron2

Name: detectron2
Version: 0.6
Summary: Detectron2 is FAIR's next-generation research platform for object detection and segmentation.
Home-page: https://github.com/facebookresearch/detectron2
Author: FAIR
Author-email: 
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: black, cloudpickle, fvcore, hydra-core, iopath, matplotlib, omegaconf, packaging, Pillow, pycocotools, tabulate, tensorboard, termcolor, tqdm, yacs
Required-by: 


In [8]:
video_path = "/home/test/drdo"

## For Local System

In [11]:
%reload_ext watermark
%watermark -v -p numpy,pandas,pycocotools,torch,torchvision,detectron2,tensorflow,deepface

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

numpy      : 1.25.2
pandas     : 2.0.3
pycocotools: 2.0.7
torch      : 2.2.1+cu121
torchvision: 0.17.1+cu121
detectron2 : 0.6
tensorflow : 2.15.0
deepface   : not installed



In [10]:
!pip install watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00


## Face Detection

In [15]:
import torch
import cv2
import numpy as np
import os
import datetime
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.model_zoo import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import ntpath
import pandas as pd
from PIL import Image
import urllib.request
import matplotlib.pyplot as plt
import itertools
import torchvision

# Load face detection model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.DEVICE = 'cpu'  # Set device to CPU
predictor = DefaultPredictor(cfg)

video_path =

def detect_faces(frame):
    outputs = predictor(frame)
    instances = outputs["instances"]
    pred_classes = instances.pred_classes if instances.has("pred_classes") else None
    boxes = instances.pred_boxes if instances.has("pred_boxes") else None

    detected_faces = []
    if pred_classes is not None and boxes is not None:
        for class_idx, box in zip(pred_classes, boxes):
            if class_idx == 0:  # 0 corresponds to 'person' class in COCO dataset
                box = box.cpu().numpy().astype(int)  # Convert box tensor to numpy array
                x0, y0, x1, y1 = box[0], box[1], box[2], box[3]
                detected_faces.append(frame[y0:y1, x0:x1])
    return detected_faces

def format_timestamp(timestamp_ms):
    # Convert timestamp (in milliseconds) to a datetime object
    timestamp = datetime.datetime.utcfromtimestamp(timestamp_ms / 1000.0).strftime("%H%M%S")
    return timestamp

def save_faces(detected_faces, output_dir, timestamp_ms, frame_number, image_count):
    # Format the timestamp to HHMMSS format
    timestamp = format_timestamp(timestamp_ms)





    for i, face in enumerate(detected_faces
                             ):
        face_name = f"{timestamp}_{frame_number}_{image_count + i}.jpg"
        save_path = os.path.join(output_dir, face_name)
        cv2.imwrite(save_path, face)
        print(f"Saved image: {face_name}")
    return len(detected_faces)

def add_frame_number(frame, frame_number):
    font = cv2.FONT_HERSHEY_SIMPLEX
    position = (10, 30)
    font_scale = 1
    font_color = (0, 255, 0)  # Green color
    line_type = 2

    cv2.putText(frame, f"Frame: {frame_number}", position, font, font_scale, font_color, line_type)

def process_video(video_path, output_dir):
    # Open video file
    cap = cv2.VideoCapture(video_path)

    frame_number = 0
    total_detected_images = 0
    frame_skip = 6  # Skip 8 frames after reading one frame

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Get timestamp of the current frame in milliseconds
        timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)

        # Read only every 7th frame
        if frame_number % 7 == 0:
            # Detect faces in the frame
            detected_faces = detect_faces(frame)

            # Save detected faces
            image_count = total_detected_images
            count = save_faces(detected_faces, output_dir, timestamp_ms, frame_number, image_count)
            total_detected_images += count

        frame_number += 1
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number * frame_skip)

    # Release video capture object
    cap.release()
    cv2.destroyAllWindows()

    return total_detected_images

# Create output directory
output_dir = "detected_faces"
os.makedirs(output_dir, exist_ok=True)

total_detected_images = process_video(video_path, output_dir)

print("Faces detected and saved successfully.")
print("Total detected images:", total_detected_images)


Faces detected and saved successfully.
Total detected images: 0


## Face Recognition

In [ ]:
from deepface import DeepFace

img1_path = r"C:\Users\madha\Documents\DRDO\test_samples\download.jpg"
img2_path = r"C:\Users\madha\Documents\DRDO\detected_faces\000040_161_35.jpg"

# Perform face verification
dfs = DeepFace.verify(img1_path=img1_path, img2_path=img2_path)

# Print the result
print(dfs)


In [ ]:
from deepface import DeepFace

models = [
  "VGG-Face",
  "Facenet",
  "Facenet512",
  "OpenFace",
  "DeepFace",
  "DeepID",
  "ArcFace",
  "SFace",
  "GhostFaceNet",
]

'''#face verification
result = DeepFace.verify(img1_path,
      img2_path,
      model_name = models[0]
)'''

#face recognition
dfs = DeepFace.find(img_path = r"C:\Users\madha\Documents\DRDO\test_samples\download-1.jpg",
db_path = r"C:\Users\madha\Documents\DRDO\detected_faces",
      model_name = models[1]
)

'''#embeddings
embedding_objs = DeepFace.represent(img_path = "img.jpg",
      model_name = models[2]
)'''

print(dfs)

In [ ]:
from deepface import DeepFace

models = [
  "VGG-Face",
  "Facenet",
  "Facenet512",
  "OpenFace",
  "DeepFace",
  "DeepID",
  "ArcFace",
  "SFace",
  "GhostFaceNet",
]

img_path = r"C:\Users\madha\Documents\DRDO\test_samples\download-1.jpg"
db_path = r"C:\Users\madha\Documents\DRDO\detected_faces"

results = {}

for model in models:
    dfs = DeepFace.find(img_path=img_path,
                        db_path=db_path,
                        model_name=model)
    results[model + "_dfs"] = dfs


In [ ]:
print(results)

In [ ]:
with open('face_recognition_results.txt', 'w') as file:
    for model_name, result in results.items():
        file.write(f"Model: {model_name}\n")
        for face in result:
            file.write(f"Face: {face}\n")
        file.write("\n")